<a href="https://colab.research.google.com/github/MarcusToledo/email-spam-classification-algorithms-comparison/blob/master/ml_spam_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Carregar o dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
column_names = [
    'word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our',
    'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail',
    'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses',
    'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit',
    'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp',
    'word_freq_hpl', 'word_freq_george', 'word_freq_650', 'word_freq_lab', 'word_freq_labs',
    'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
    'word_freq_technology', 'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
    'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 'word_freq_re',
    'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'char_freq_;', 'char_freq_(',
    'char_freq_[', 'char_freq_!', 'char_freq_$', 'char_freq_#', 'capital_run_length_average',
    'capital_run_length_longest', 'capital_run_length_total', 'label'
]
data = pd.read_csv(url, header=None, names=column_names)

# Selecionando os atributos
X = data.iloc[:, :-1].values  # Vou usar todos, o mínimo seria 7
y = data.iloc[:, -1].values  # Selecionado a última coluna 'label'

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_ize=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Lista de modelos
models = {
    "Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(12, 8), max_iter=1000, verbose=False)
}

# Armazenar os resultados
results = []

# Treinar e avaliar cada modelo
for model_name, model in models.items():
    model.fit(X_train, y_train)

    # Fazer previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Avaliar o modelo
    accuracy = accuracy_score(y_test, y_pred) * 100.0
    accuracy_non_normalized = accuracy_score(y_test, y_pred, normalize=False)
    report = classification_report(y_test, y_pred, output_dict=True)
    precision_avg = report['weighted avg']['precision']
    recall_avg = report['weighted avg']['recall']
    f1_score_avg = report['weighted avg']['f1-score']
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Calcular True Positive Rate(TPR) e True Negative Rate(TNR)

    # TN -> True Negatives -> Número de verdadeiros negativos (Casos negativos
    # identificados como negativos)

    # TP -> True Positives -> Número de verdadeiros positivos (Casos positivos
    # identificados como positivos)

    # FP -> False Positives -> Número de falsos positivos (Casos negativos
    # classificados como positivos)

    # FN -> False Negatives -> Número de falsos negativos (Casos positivos
    # classificados como negativos)
    TN, FP, FN, TP = conf_matrix.ravel()
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)

    # Formatar a matriz de confusão
    conf_matrix_df = pd.DataFrame(conf_matrix, index=['Real: Não-Spam', 'Real: Spam'], columns=['Previsto: Não-Spam', 'Previsto: Spam'])

    # Calcular a pontuação composta
    composed_score = (accuracy + precision_avg * 100 + recall_avg * 100 + f1_score_avg * 100 + TPR * 100 + TNR * 100) / 6

    # Armazenar os resultados
    results.append({
        "modelo": model_name,
        "acuracia": accuracy,
        "qt_acuracia": accuracy_non_normalized,
        # "precision_avg": precision_avg,
        # "recall_avg": recall_avg,
        # "f1_score_avg": f1_score_avg,
        # "TPR": TPR,
        # "TNR": TNR,
        "composed_score": composed_score,
        "report": report,
        "matrix": conf_matrix_df,
        "y_test_shape": y_test.shape[0],
    })

In [ ]:
# Ordenar os resultados por maior pontuação composta
results = sorted(results, key=lambda x: x["composed_score"], reverse=True)

# Exibir os resultados ordenados
for result in results:
    print(f"Modelo: {result['modelo']}")
    print(f"Acurácia: {result['acuracia']:.2f}%")
    print(f"Acurácia (quantidade amostras): {result['qt_acuracia']} / {result['y_test_shape']}")
    # print(f"Precisão Média: {result['precision_avg']:.2f}")
    # print(f"Recall Médio: {result['recall_avg']:.2f}")
    # print(f"F1-Score Médio: {result['f1_score_avg']:.2f}")
    # print(f"Taxa de Verdadeiro Positivo (TPR): {result['TPR']:.2f}")
    # print(f"Taxa de Verdadeiro Negativo (TNR): {result['TNR']:.2f}")
    print(f"Pontuação Composta: {result['composed_score']:.2f}")
    print("Relatório de Classificação:")
    print(pd.DataFrame(result["report"]).transpose())
    print("Matriz de Confusão:")
    print(result["matrix"])
    print("\n" + "="*60 + "\n")

Modelo: Random Forest
Acurácia: 95.66%
Acurácia (quantidade amostras): 881 / 921
Pontuação Composta: 95.52
Relatório de Classificação:
              precision    recall  f1-score     support
0              0.945554  0.981168  0.963031  531.000000
1              0.972973  0.923077  0.947368  390.000000
accuracy       0.956569  0.956569  0.956569    0.956569
macro avg      0.959263  0.952122  0.955200  921.000000
weighted avg   0.957164  0.956569  0.956399  921.000000
Matriz de Confusão:
                Previsto: Não-Spam  Previsto: Spam
Real: Não-Spam                 521              10
Real: Spam                      30             360


Modelo: Neural Network
Acurácia: 93.70%
Acurácia (quantidade amostras): 863 / 921
Pontuação Composta: 93.53
Relatório de Classificação:
              precision    recall  f1-score     support
0              0.927667  0.966102  0.946494  531.000000
1              0.951087  0.897436  0.923483  390.000000
accuracy       0.937025  0.937025  0.937025    0.9